# Interface - *neutralizing-bias*

#### Run in Shell
__Concurrent__

`python joint/inference.py --bert_encoder --bert_full_embeddings --coverage --debias_checkpoint models/debias_model.ckpt --debias_weight 1.3 --inference_output inference_concurrent/output.txt --no_tok_enrich --pointer_generator --test inference_concurrent/data.txt --working_dir inference_concurrent/`

__Modular__

`python joint/inference.py --activation_hidden --bert_full_embeddings --checkpoint models/model.ckpt --coverage --debias_weight 1.3 --extra_features_top --inference_output inference_modular/output.txt --pointer_generator --pre_enrich --test inference_modular/data.txt --token_softmax --working_dir inference_modular/`

_Optional:_ `--categories_file bias_data/WNC/revision_topics.csv --category_input --tagging_pretrain_epochs 3 --test_batch_size 1`

In [2]:
import os
import re
import ast
import sys
import subprocess
import spacy
from typing import Tuple, List, Dict, Union
from pytorch_pretrained_bert.tokenization import BertTokenizer

NLP = spacy.load(
    "en_core_web_sm"
)  # run "python -m spacy download en_core_web_sm" to initially download the model
TOKENIZER = BertTokenizer.from_pretrained("bert-base-uncased",
                                          cache_dir="cache")


/home/pp/master-thesis/neutralizing-bias/.venv-nb/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
06/19/2022 13:42:18 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [3]:
def get_pos_dep(s: str) -> Tuple[str, str]:
    """Get POS and dependency tags for a given string.
    """
    toks = s.split()

    def words_from_toks(toks):
        words = []
        word_indices = []
        for i, tok in enumerate(toks):
            if tok.startswith('##'):
                words[-1] += tok.replace('##', '')
                word_indices[-1].append(i)
            else:
                words.append(tok)
                word_indices.append([i])
        return words, word_indices

    out_pos, out_dep = [], []
    words, word_indices = words_from_toks(toks)
    analysis = NLP(" ".join(words))

    if len(analysis) != len(words):
        return None, None

    for analysis_tok, idx in zip(analysis, word_indices):
        out_pos += [analysis_tok.pos_] * len(idx)
        out_dep += [analysis_tok.dep_] * len(idx)

    assert len(out_pos) == len(out_dep) == len(toks)

    return " ".join(out_pos), " ".join(out_dep)

In [4]:
def tokenize(s: str):
    """BERT-tokenize a given string.
    """
    global TOKENIZER
    tok_list = TOKENIZER.tokenize(s.strip())
    return " ".join(tok_list)

In [48]:
def debias(
    input: List[str],
    concurrent: bool = False,
) -> List[Dict[str, Union[str, float]]]:
    """
    Adapter for running the de-biasing script "inference.py" on the concurrent and the modular model architecture.

    Args:
        input - list of sentences to be debiased
        concurrent - run concurrent model (if False, run modular model)

    Returns:
        results - list of output dicts
    """
    assert len(input) > 0

    working_dir = "inference_concurrent" if concurrent else "inference_modular"
    inference_output = os.path.join(working_dir, "output.txt")
    test = os.path.join(working_dir, "data.txt")

    if os.path.exists(inference_output):
        os.remove(inference_output)

    if os.path.exists(test):
        os.remove(test)

    results = []

    # create input file
    with open(test, "w") as f:
        for s in input:
            tok = tokenize(s)
            pos, dep = get_pos_dep(tok)
            f.write("0\t" + (tok + "\t") * 2 + (s + "\t") * 2 + pos + "\t" +
                    dep + "\n")

    # compile command line arguments
    args = [
        sys.executable, "joint/inference.py", "--bert_full_embeddings",
        "--coverage", "--debias_weight", "1.3", "--inference_output",
        inference_output, "--pointer_generator", "--test", test,
        "--working_dir", working_dir
    ]

    if concurrent:
        args.append("--bert_encoder")
        args.append("--debias_checkpoint")
        args.append("models/debias_model.ckpt")
        args.append("--no_tok_enrich")

    else:
        args.append("--activation_hidden")
        args.append("--checkpoint")
        args.append("models/model.ckpt")
        args.append("--extra_features_top")
        args.append("--pre_enrich")
        args.append("--test_batch_size")
        args.append("1")
        args.append("--token_softmax")

    # execute shell script
    subprocess.run(" ".join(args), shell=True)
    
    ## use for debugging ##
    # debug_output = subprocess.run(" ".join(args),
    #                               text=True,
    #                               capture_output=True)
    # print(debug_output.stderr)
    # print(debug_output.stdout)
    #######################

    # read output
    with open(inference_output, "r") as f:
        lines = f.readlines()
        for line in lines:
            if re.match(r'^IN SEQ:', line):
                in_seq_tok = line.split("\t")[1][3:-2]
                in_seq = in_seq_tok.replace(" ##", "")
                in_seq = re.sub(r'\s([.,;?!"])', r'\1', in_seq)

            if re.match(r'^PRED SEQ:', line):
                pred_seq_tok = line.split("\t")[1][3:-2]
                pred_seq = pred_seq_tok.replace(" ##", "")
                pred_seq = re.sub(r'\s([.,;?!"])', r'\1', pred_seq)

            if re.match(r'^PRED DIST:', line):
                pred_dist = line.split("\t")[1]
                pred_dist = ast.literal_eval(pred_dist)

                # create sequence with underlined changes
                change_seq = in_seq_tok.split()
                difs = list(
                    set(in_seq_tok.split()) - set(pred_seq_tok.split()))

                for dif in difs:
                    idx = change_seq.index(dif)

                    if pred_dist[idx] > 0.1:
                        change_seq[idx] = "\033[4m{}\033[0m".format(
                            change_seq[idx])

                change_seq = " ".join(change_seq)
                change_seq = change_seq.replace(" ##", "")
                change_seq = re.sub(r'\s([.,;?!"])', r'\1', change_seq)

                results.append({
                    "change_seq": change_seq,
                    "in_seq": in_seq,
                    "pred_seq": pred_seq,
                    "in_seq_tok": in_seq_tok,
                    "pred_seq_tok": pred_seq_tok,
                    "pred_dist": pred_dist,
                })

    return results


#### 1. Define the input sentences

In [30]:
inputs = [
    "jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other dominant peoples, religions and cultures.",
    "Black people are always criminal."
]

#### 2. Run the model

In [31]:
results_modular = debias(inputs)

06/13/2022 10:47:40 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at inference_modular/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2it [00:00, 2638.76it/s]


SKIPPED  0


/home/pp/master-thesis/neutralizing-bias/.venv-nb/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
06/13/2022 10:47:42 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at inference_modular/cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
06/13/2022 10:47:42 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file inference_modular/cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpj09v6h8b
06/13/2022 10:47:45 - INFO - pytorch_pretrained_bert.

LOADING FROM models/model.ckpt
...DONE


100%|██████████| 2/2 [00:24<00:00, 12.27s/it]


Change lines 75 and 80 in `joint/inference.py` to toggle GPU/CPU use

In [49]:
results_concurrent = debias(inputs, concurrent=True)

06/13/2022 11:06:45 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at inference_concurrent/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2it [00:00, 2556.72it/s]


SKIPPED  0


/home/pp/master-thesis/neutralizing-bias/.venv-nb/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
06/13/2022 11:06:47 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at inference_concurrent/cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
06/13/2022 11:06:47 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file inference_concurrent/cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpuzsk8ytu
06/13/2022 11:06:50 - INFO - pytorch_pretrained

LOADING DEBIASER FROM models/debias_model.ckpt
DONE.


100%|██████████| 1/1 [00:33<00:00, 33.97s/it]


#### 3. Print the results

In [33]:
for result in results_modular:
    print("Input:   {}".format(result["in_seq"]))
    print("Changes: {}".format(result["change_seq"]))
    print("Output:  {}".format(result["pred_seq"]))
    print("Bias scores: " + ("{} ({:.3f}) " * len(result["pred_dist"])).format(
        *tuple([
            tok
            for dist in zip(result["in_seq_tok"].split(), result["pred_dist"])
            for tok in dist
        ])))
    print()

Input:   jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other dominant peoples, religions and cultures.
Changes: jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other dominant peoples, religions and cultures.
Output:  jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other peoples, religions and cultures.
Bias scores: jewish (0.015) history (0.000) is (0.014) the (0.001) history (0.002) of (0.000) the (0.001) jewish (0.130) people (0.014) , (0.000) their (0.202) religion (0.002) , (0.000) and (0.000) culture (0.000) , (0.000) as (0.000) it (0.000) developed (0.002) and (0.000) interact (0.002) ##ed (0.001) with (0.000) other (0.024) dominant (0.445) peoples (0.002) , (0.000) religions (0.000) and (0.000) cultures (0.000) . (0.000) 

Input:   black people are always criminal.
Changes:

In [50]:
for result in results_concurrent:
    print("Input:   {}".format(result["in_seq"]))
    print("Changes: {}".format(result["change_seq"]))
    print("Output:  {}".format(result["pred_seq"]))
    print()

Input:   jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other dominant peoples, religions and cultures.
Changes: jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other dominant peoples, religions and cultures.
Output:  jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other dominant peoples, religions and cultures.

Input:   black people are always criminal.
Changes: black people are always criminal.
Output:  black people are sometimes criminal.



In [ ]:
def gradient(x: float, start_col: Tuple[int]=(255, 255, 255), end_col: Tuple[int]=(250, 50, 50)) -> str:
    """ Returns the HEX code for a color at position x ∈ [0; 1] within a color gradient of start_col and end_col.
    """
    rgb = (
        int((1 - x) * start_col[0] + x * end_col[0]),
        int((1 - x) * start_col[1] + x * end_col[1]),
        int((1 - x) * start_col[2] + x * end_col[2]))
    return "#%02x%02x%02x" % rgb


def print_results(results: List[Dict[str, Union[str, List[float]]]], out_file: str="") -> None:
    """ Prints the result tokens with a highlighted background according to the subjective bias probability.
    """
    from IPython.display import display, HTML

    tok_lists = [x["in_seq_tok"] for x in results]
    dist_lists = [x["pred_dist"]for x in results]
    html_list = []

    for toks, dists in zip(tok_lists, dist_lists):
        html_string = "<div style='background-color:white;padding:10px;margin:-8px'>"

        for tok, dist in zip(toks.split(" "), dists):
            html_string += "<span style='color:black;background-color: " + gradient(dist) + "'>" + tok + "</span>" + " "

        html_string += "</div>"
        html_list.append(HTML(html_string))

    if len(out_file) > 0:
        with open(out_file, "w") as f:
            f.write("\n".join(html.data) + "\n")

    for html in html_list:
        display(html)


print_results(results_modular)

In [24]:
def generate_output(
    in_file: str,
    out_file: str,
    html: bool=False,
    html_file: str=None
    ) -> None:
    """ Generates an output and optionally an HTML file.
    """
    # read input
    results = []
    with open(in_file, "r") as f:
        lines = f.readlines()

        for line in lines:
            if re.match(r'^IN SEQ:', line):
                in_seq_tok = line.split("\t")[1][3:-2]
                in_seq = in_seq_tok.replace(" ##", "")
                in_seq = re.sub(r'\s([.,;?!"])', r'\1', in_seq)

            if re.match(r'^PRED SEQ:', line):
                pred_seq_tok = line.split("\t")[1][3:-2]
                pred_seq = pred_seq_tok.replace(" ##", "")
                pred_seq = re.sub(r'\s([.,;?!"])', r'\1', pred_seq)

            if re.match(r'^PRED DIST:', line):
                pred_dist = line.split("\t")[1]
                pred_dist = ast.literal_eval(pred_dist)

                results.append({
                    "in_seq_tok": in_seq_tok,
                    "pred_seq": pred_seq,
                    "pred_dist": pred_dist,
                })

    # write output
    with open(out_file, "w") as f:
        text = "\n".join([result["pred_seq"] for result in results]) + "\n"
        text = text.replace(" ' ", "'")
        text = text.replace(" / ", "/")
        text = text.replace(" & ", "&")
        text = text.replace(" \' ", "'")
        text = text.replace(" \'", "'")
        text = text.replace("( ", "(")
        text = text.replace(" )", ")")
        text = text.replace(" :", ":")
        f.write(text)

    # write HTML output
    if html:
        html_list = []
        tok_lists = [x["in_seq_tok"] for x in results]
        dist_lists = [x["pred_dist"]for x in results]

        for toks, dists in zip(tok_lists, dist_lists):
            html_string = "<div style='background-color:white;padding:10px;margin:-8px'>"

            for tok, dist in zip(toks.split(" "), dists):
                html_string += "<span style='color:black;background-color: " + gradient(dist) + "'>" + tok + "</span>" + " "

            html_string += "</div>"
            html_list.append(html_string)

        with open(html_file, "w") as f:
            f.write("\n".join(html_list) + "\n")

In [25]:
generate_output(
    in_file="/home/pp/master-thesis/neutralizing-bias/src/inference/modular/results_modular.txt",
    out_file="/home/pp/master-thesis/neutralizing-bias/src/inference/modular/output_modular.txt",
    html=True,
    html_file="/home/pp/master-thesis/neutralizing-bias/src/inference/output_modular.html"
)

generate_output(
    in_file="/home/pp/master-thesis/neutralizing-bias/src/inference/concurrent/results_concurrent.txt",
    out_file="/home/pp/master-thesis/neutralizing-bias/src/inference/concurrent/output_concurrent.txt",
)

for i in range(1, 11):
    generate_output(
        in_file="/home/pp/master-thesis/neutralizing-bias/src/inference/concurrent/iter_10/results_concurrent_" + str(i) + ".txt",
        out_file="/home/pp/master-thesis/neutralizing-bias/src/inference/concurrent/iter_10/output_concurrent_" + str(i) + ".txt",
    )

    generate_output(
        in_file="/home/pp/master-thesis/neutralizing-bias/src/inference/modular/iter_10/results_modular_" + str(i) + ".txt",
        out_file="/home/pp/master-thesis/neutralizing-bias/src/inference/modular/iter_10/output_modular_" + str(i) + ".txt",
        html=True,
        html_file="/home/pp/master-thesis/neutralizing-bias/src/inference/modular/iter_10/output_modular_" + str(i) + ".html"
    )

generate_output(
    in_file="/home/pp/master-thesis/neutralizing-bias/src/inference/modular/strap_modular/results_strap_modular.txt",
    out_file="/home/pp/master-thesis/neutralizing-bias/src/inference/modular/strap_modular/output_strap_modular.txt",
)

generate_output(
    in_file="/home/pp/master-thesis/neutralizing-bias/src/inference/concurrent/strap_concurrent/results_strap_concurrent.txt",
    out_file="/home/pp/master-thesis/neutralizing-bias/src/inference/concurrent/strap_concurrent/output_strap_concurrent.txt",
)